# High-level Gluon Example

In [ ]:
import os
import sys
import numpy as np
import math
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
from common.params import *
from common.utils import *

In [ ]:
ctx = mx.gpu()

In [ ]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("MXNet: ", mx.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())

In [ ]:
def SymbolModule():
    sym = gluon.nn.Sequential()
    with sym.name_scope():
        sym.add(gluon.nn.Conv2D(channels=50, kernel_size=3, padding=1, activation='relu'))
        sym.add(gluon.nn.Conv2D(channels=50, kernel_size=3, padding=1))
        sym.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        sym.add(gluon.nn.LeakyReLU(0))
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Conv2D(channels=100, kernel_size=3, padding=1, activation='relu'))
        sym.add(gluon.nn.Conv2D(channels=100, kernel_size=3, padding=1))
        sym.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        sym.add(gluon.nn.LeakyReLU(0))
        sym.add(gluon.nn.Dropout(0.25))
        # feature map size is 8*8 by pooling
        sym.add(gluon.nn.Dense(512, activation='relu'))
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Dense(N_CLASSES))
    return sym

In [ ]:
def init_model(m):
    # Implementation of momentum:
    # v = \rho * v + g \\
    # p = p - lr * v
    trainer = gluon.Trainer(m.collect_params(), 'sgd',
                    {'learning_rate': LR, 'momentum':MOMENTUM})
    criterion = gluon.loss.SoftmaxCrossEntropyLoss()
    return trainer, criterion

In [ ]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)
# Torch-specific
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

In [ ]:
%%time
sym = SymbolModule()
sym.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

In [ ]:
%%time
trainer, criterion = init_model(sym)

In [ ]:
%%time
# Sets training = True 
for j in range(EPOCHS):
    for data, target in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        # Get samples
        data = nd.array(data).as_in_context(ctx)
        target = nd.array(target).as_in_context(ctx)
        with autograd.record():
            # Forwards
            output = sym(data)
            # Loss
            loss = criterion(output, target)
        # Back-prop
        loss.backward()
        trainer.step(data.shape[0])
    # Log
    print(j)

In [ ]:
%%time
# Test model
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, target in yield_mb(x_test, y_test, BATCHSIZE):
    # Get samples
    data = nd.array(data).as_in_context(ctx)
    # Forwards
    output = sym(data)
    pred = nd.argmax(output, axis=1)
    # Collect results
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = pred.asnumpy()
    c += 1

In [ ]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))